<a href="https://colab.research.google.com/github/kevinhhl/options-pricing-tools-and-trading-strategies/blob/main/Black_Scholes_Merton_Model_Part1_Screening_YF_for_theoretical_edges.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##### Imports

In [1]:
!pip install yahoo_fin

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
!pip install yfinance

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
import math
from scipy.stats import norm
import pandas as pd
from pandas import DataFrame
from yahoo_fin import options
from datetime import date
import yfinance as yf


---
# **Overview**


**Black-Scholes-Merton Model ("BSM") - Part 1**
<br>
This script will allow us to obtain options data provided by Yahoo Finance, and then for each quoted contract, to compute the theoretical value of it by applying the BSM. The goal is to find the difference between each contract’s quoted price and its theoretical value to allow us to ask further questions with regards to whether there will possibly be theoretical edges, especially with contracts associated with having the largest theoretical price discrepancies.
<br>

**Disadvantages to BSM** are, such as, but not limited to:
* Was modeled for European-style options; early-exercise is not allowed
* Assumes price to follow a random walk according to a brownian motion with a constant drift. <br>(I have created a separate notebook to explore this topic. [Link here.](https://github.com/kevinhhl/portfolio-management-tools/blob/main/Monte_Carlo_Simulation_Random_Walk.ipynb))
* Ignores dividends
* Inputs can be subjective, especially for expected volatility
* Taxes and transaction costs are ignored

**Advantages to BSM**
* It is widely used
* It's simple to apply

<br>

**Black-Scholes-Merton Model (“BSM”) - Part 2**
There will be another part to this project in "Part 2". Next time, position analysis will be the focus. We will find ways to present multi-legged spreads in table formats to summarize the common Greeks.

# **Understanding the Model:**

The derivation of the BSM is rather complex. Through research [1], I use my own words to summarize how I understand it from practical perspective.

The premise of the Black-Scholes Model contains two components. It can be described as taking the difference of: <br>
* (a) the expected value of a stock in the event that it reaches above/below exercise price on the date of expiration for call/put options, respectively, and 
* (b) the expected payout at the exercise price

>* These components are expressed as:
<br>\begin{equation}
TV_{call}=se^{rt}N(d_{1})-xN(d_{2})
\end{equation}
<br>\begin{equation}
TV_{put}=xe^{-rt}N(-d_{2})-sN(-d_{1})
\end{equation}
> 
> **Where**:
>* TV is the theoretical value
* s is the spot price at the current moment
* x is the exercise price of the option
* t is the time till maturity in no. of years
* r is the risk free rate per annum
* σ is the annualized volatility
* N(d) is the CDF of a normal distribution.
<br>
<br>\begin{equation}
d_{1} =\frac{ln(\frac{S}{X})+(r+\frac{\sigma}{2}^2)t}{\sigma\sqrt{t}}
\end{equation}
<br>\begin{equation}
d_{2}=d_{1}-\sigma\sqrt{t}
\end{equation}


*Contemplative observations:*


* The probability of price to be in-the-money for call/put options can be pictured as if seeing the data points '*d*' landing in the right/left sides of the CDF (for call/put options, respectively). 
* Probability of the put option to be in-the-money would be N(-d), which represents the left tail of the CDF; speaking of the area from negative-infinity to 0-d. The probability of a call option to be in-the-money would be 1-N(-d); being the right-tail of the CDF.
* Price is not normally distributed. Instead, a lognormal distribution would better describe it. We still want to associate probability of price reaching X with the normal distribution, so we need to make the adjustments to *d*. Also, we will want to risk-adjust *d* according to *r*

<br>

---
*References:*

[1] Natenberg, Sheldon. <i>Chapter 18, Option Volatility and Pricing, Second Edition</i>. McGraw-Hill Edu., 2015.



# **Implementation:**

In [4]:
class BSM:
  
  def __init__(self, s,t,r,sigma):
    # Input(s) that are not set by constructor: 
    # self.x - changes as we loop through the options chain provided by Yahoo Finance

    # inputs do not change:
    self.s = s
    self.t = t
    self.r = r
    self.sigma = sigma

  def set_x(self, x):
    self.x = x
    self._recalc()

  def _recalc(self):
    # When input variable 'x' changes, will need to recalc
    self.d1 = self._d1() 
    self.d2 = self.d1 - self.sigma * math.sqrt(self.t)
    self.tv_call = self.s * norm.cdf(self.d1) - self.x*math.exp(-self.r*self.t)*norm.cdf(self.d2)
    self.tv_put = self.x * math.exp(-self.r*self.t)-self.s+self.tv_call

  def _d1(self):
    a = math.log(self.s/ self.x)
    b = (self.r+self.sigma**2/2)*self.t
    return(a+b)/self.sigma*math.sqrt(self.t)


---
Checking calculations with @YuChenAmberLu's version in [<link\>](https://github.com/YuChenAmberLu/Options-Calculator)

In [5]:
# Validations with YuChenAmberLu's version:
test_model = BSM(s=100,t=0.128767,r=0.2,sigma=0.2)
test_model.set_x(100)
assert (test_model.tv_call-4.112199).round(5)==0
assert (test_model.tv_put-1.569736).round(5)==0

---
# **Application:**
> **Comparing with Yahoo Finance**
>
> The idea of this project is to loop through certain option chains in Yahoo and see if we can scan for options with theoretical edges. This notebook is a mere illustration of how it can be done, we will only calculate TSLA's calls that are near at-the-money. 

## 😎 Setting up our model:

In [6]:
# Manual inputs:
ticker              = "TSLA"          # <-- Symbol
r                   = .0512           # <-- The continuously compounded risk-free rate
sigma               = 0.50            # <-- Expected annualized volatility
date_today          = date(2023,2,18) # <-- Today's date
date_expire         = date(2023,2,24) # <-- Expiration date (Must be according to a valid contract)

In [7]:
# Confirming that the expiration date is valid.
exp_dates = options.get_expiration_dates(ticker)
date_expire_str         = date_expire.strftime("%B %d, %Y") 
assert date_expire_str in exp_dates
exp_dates

['February 17, 2023',
 'February 24, 2023',
 'March 3, 2023',
 'March 10, 2023',
 'March 17, 2023',
 'March 24, 2023',
 'March 31, 2023',
 'April 21, 2023',
 'May 19, 2023',
 'June 16, 2023',
 'July 21, 2023',
 'September 15, 2023',
 'December 15, 2023',
 'January 19, 2024',
 'March 15, 2024',
 'June 21, 2024',
 'September 20, 2024',
 'January 17, 2025',
 'June 20, 2025']

In [8]:
# Inputs that can be obtained programmatically
s = (yf.Ticker(ticker).info["bid"]+yf.Ticker(ticker).info["ask"])/2
t = (date_expire-date_today).days/365

In [9]:
# helper function
def _print(option_type, model):
  print("{}, close={}; {}; expiring on {}".format(ticker, round(s,2), option_type, date_expire_str))
  print("Holding all variables constant, quoted price vs. theoretical values (TV) as follows:\n")

  chain = options.get_options_chain(ticker, date_expire_str)[option_type]
  for i in range(len(chain)):
    x = chain["Strike"][i]
    model.set_x(x)
    q = chain["Last Price"][i]
    
    tv = None
    if option_type == "calls":
      tv = model.tv_call
    elif option_type == "puts":
      tv = model.tv_put
    if tv > 0: 
      print("{}: strike= {},\tquote={}\t\tTV= {};\t\tdiff.= {}".format(option_type, x, q, tv.round(2), (q-tv).round(2)))

---

###**🟢 Call options:**

In [10]:
model = BSM(s=s,t=t,r=r,sigma=sigma)
_print("calls", model)

TSLA, close=201.79; calls; expiring on February 24, 2023
Holding all variables constant, quoted price vs. theoretical values (TV) as follows:

calls: strike= 15.0,	quote=197.85		TV= 139.98;		diff.= 57.87
calls: strike= 20.0,	quote=108.18		TV= 131.98;		diff.= -23.8
calls: strike= 25.0,	quote=111.18		TV= 125.05;		diff.= -13.87
calls: strike= 30.0,	quote=104.87		TV= 118.85;		diff.= -13.98
calls: strike= 55.0,	quote=141.75		TV= 93.97;		diff.= 47.78
calls: strike= 60.0,	quote=140.33		TV= 89.75;		diff.= 50.58
calls: strike= 65.0,	quote=129.26		TV= 85.7;		diff.= 43.56
calls: strike= 70.0,	quote=104.95		TV= 81.8;		diff.= 23.15
calls: strike= 75.0,	quote=90.45		TV= 78.04;		diff.= 12.41
calls: strike= 80.0,	quote=131.15		TV= 74.39;		diff.= 56.76
calls: strike= 85.0,	quote=115.75		TV= 70.85;		diff.= 44.9
calls: strike= 90.0,	quote=111.76		TV= 67.4;		diff.= 44.36
calls: strike= 95.0,	quote=108.13		TV= 64.04;		diff.= 44.09
calls: strike= 98.0,	quote=74.5		TV= 62.07;		diff.= 12.43
calls: strike= 99.

---
###**🔴 Put options:**

In [11]:
model = BSM(s=s,t=t,r=r,sigma=sigma)
_print("puts", model)

TSLA, close=201.79; puts; expiring on February 24, 2023
Holding all variables constant, quoted price vs. theoretical values (TV) as follows:

puts: strike= 192.5,	quote=3.4		TV= 0.24;		diff.= 3.16
puts: strike= 195.0,	quote=4.15		TV= 1.53;		diff.= 2.62
puts: strike= 197.5,	quote=5.2		TV= 2.83;		diff.= 2.37
puts: strike= 200.0,	quote=6.15		TV= 4.13;		diff.= 2.02
puts: strike= 202.5,	quote=7.45		TV= 5.44;		diff.= 2.01
puts: strike= 205.0,	quote=8.85		TV= 6.76;		diff.= 2.09
puts: strike= 207.5,	quote=10.5		TV= 8.08;		diff.= 2.42
puts: strike= 210.0,	quote=12.13		TV= 9.41;		diff.= 2.72
puts: strike= 212.5,	quote=13.92		TV= 10.74;		diff.= 3.18
puts: strike= 215.0,	quote=15.85		TV= 12.08;		diff.= 3.77
puts: strike= 217.5,	quote=18.4		TV= 13.43;		diff.= 4.97
puts: strike= 220.0,	quote=20.35		TV= 14.78;		diff.= 5.57
puts: strike= 222.5,	quote=22.4		TV= 16.14;		diff.= 6.26
puts: strike= 225.0,	quote=24.95		TV= 17.5;		diff.= 7.45
puts: strike= 227.5,	quote=27.11		TV= 18.87;		diff.= 8.24
puts: st